## 6-1. 코랩 노트북
- 모델을 파인튜닝하려면 하드웨어 가속기를 사용해야 계산 속도를 높일 수 있습니다.
- 코랩에서는 GPU와 TPU 두 종류의 가속기를 지원합니다.
- 코랩 화면의 연결 GPU 메뉴 탭에서 런타임 > 런타임 유형 변경을 클릭합니다.
- 이후 GPU 혹은 TPU 둘 중 하나를 선택합니다.

* 파인튜닝(fine-tuning)은 기계 학습 모델을 특정 작업이나 데이터에 더 잘 맞도록 조정하는 과정을 의미.
주로 미리 학습된(pre-trained) 모델을 가져와서 새로운 작업에 맞게 조정하는 방식으로 사용

## 6-2. 각종 설정하기
### 1. TPU 관련 패키지를 설치 -> GPU 사용 (할 필요 없음)
TPU 관련 패키지를 설치합니다. TPU 사용시 아래 라인 첫 문자(#)를 지우고 수행하세요. GPU를 쓴다면 아래 라인을 실행할 필요가 없습니다.

In [ ]:
# 코드 6-1 TPU 관련 패키지 설치
# TPU 사용시 아래 라인 첫 문자(#)를 지우고 수행하세요.
# pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

####2. 의존성 패키지 설치
TPU 이외에 의존성 있는 패키지를 설치합니다.

In [ ]:
# 코드 6-2 의존성 패키지 설치
!pip install ratsnlp

####3. 구글 드라이브 연동하기
모델 체크포인트 등을 저장해 둘 구글 드라이브를 연결
자신의 구글 계정에 적용됩니다.

In [ ]:
# 코드 6-3 구글드라이브와 연결
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

#### 4. 각종 설정
모델 하이퍼파라메터(hyperparameter)와 저장 위치 등 설정 정보를 선언합니다.
- PyTorch 사용
- kcbert-base 모델을 한국해양대학교 자연언어처리 연구실에서 공개한 데이터와 자체 제작한 데이터를 합쳐 사용.

참고로 TrainArguments의 각 인자(argument)가 하는 역할과 의미는 다음과 같습니다.

- pretrained_model_name : 프리트레인 마친 언어모델의 이름(단 해당 모델은 허깅페이스 라이브러리에 등록되어 있어야 합니다)
- downstream_corpus_name : 다운스트림 데이터의 이름.
- downstream_model_dir : 파인튜닝된 모델의 체크포인트가 저장될 위치. /gdrive/My Drive/nlpbook/checkpoint-paircls라고 함은 자신의 구글 드라이브의 내 폴더 하위의 nlpbook/checkpoint-paircls 디렉토리에 모델 체크포인트가 저장됩니다.
- batch_size : 배치 크기. 하드웨어 가속기로 GPU를 선택(torch.cuda.is_available() == True)했다면 32, TPU라면(torch.cuda.is_available() == False) 4. 코랩 환경에서 TPU는 보통 8개 코어가 할당되는데 batch_size는 코어별로 적용되는 배치 크기이기 때문에 이렇게 설정해 둡니다.
- learning_rate : 러닝레이트. 1회 스텝에서 한 번에 얼마나 업데이트할지에 관한 크기를 가리킵니다. 이와 관련한 자세한 내용은 3-2-2장 Technics를 참고하세요.
- max_seq_length : 토큰 기준 입력 문장 최대 길이. 이보다 긴 문장은 max_seq_length로 자르고, 짧은 문장은 max_seq_length가 되도록 스페셜 토큰(PAD)을 붙여 줍니다.
- epochs : 학습 에폭 수. 5이라면 학습 데이터를 5회 반복 학습합니다.
- tpu_cores : TPU 코어 수. 하드웨어 가속기로 GPU를 선택(torch.cuda.is_available() == True)했다면 0, TPU라면(torch.cuda.is_available() == False) 8.
- seed : 랜덤 시드(정수, integer). None을 입력하면 랜덤 시드를 고정하지 않습니다.

In [ ]:
# 코드 6-4 모델 환경 설정
import torch
from ratsnlp.nlpbook.ner import NERTrainArguments
args = NERTrainArguments(
    pretrained_model_name="beomi/kcbert-base", # 프리트레인 마친 언어모델의 이름
    downstream_corpus_name="ner", # 다운스트림 데이터의 이름.
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-ner", # 파인튜닝된 모델의 체크포인트가 저장될 위치
    batch_size=32 if torch.cuda.is_available() else 4,  # batch_size
    learning_rate=5e-5, # learning rate
    max_seq_length=64, # 토큰 기준 입력 문장 최대 길이
    epochs=3, # 학습 횟수
    tpu_cores=0 if torch.cuda.is_available() else 8, # TPU 코어 수
    seed=7, # 랜덤 시드
)

####5. 랜덤 시드 고정
- 학습 재현을 위해 랜덤 시드를 고정합니다.
- args에 지정된 시드로 고정하는 역할

Tip: 난수는 배치(batch)를 뽑거나 드롭아웃 대상 뉴런의 위치를 정할 때 등 다양하게 쓰입니다.
컴퓨터는 난수 생성 알고리즘을 사용해 난수를 만들어내는데요. 이 때 난수 생성 알고리즘을 실행하기 위해 쓰는 수를 랜덤 시드라고 합니다.
만일 같은 시드를 사용한다면 컴퓨터는 계속 같은 패턴의 난수를 생성하게 됩니다.

In [ ]:
# 코드 6-5 랜덤 시드 고정
from ratsnlp import nlpbook
nlpbook.set_seed(args)

####6. 로거 설정
- 메세지 출력 등을 위한 logger를 설정합니다.
- 각종 로그들을 출력하는 로거를 설정

In [ ]:
# 코드 6-6 로거 설정
nlpbook.set_logger(args)

###6-3. 말뭉치 다운로드
- 튜토리얼 데이터 다운로드를 수행합니다.
- 데이터를 내려받는 도구로 nlpbook에 포함된 패키지를 사용해, 코랩 로컬 환경에 corpus_name(ner)에 해당하는 말뭉치를 내려받습니다.





In [ ]:
# 코드 6-7 말뭉치 다운로드
nlpbook.download_downstream_dataset(args)

### 6-4. 토크나이저 준비
- 토큰화를 수행하는 토크나이저를 선언합니다.
- 토크나이저(tokenizer)는 토큰화를 수행하는 프로그램이라는 뜻입니다.

In [ ]:
# 코드 6-8 토크나이저 준비
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

### 6-5. 데이터 전처리 하기
- 딥러닝 모델을 학습하려면 학습데이터를 배치(batch) 단위로 지속적으로 모델에 공급해 주어야 합니다.
- 파이토치(PyTorch)에서는 이 역할을 데이터 로더(DataLoader)가 수행합니다. 이건 TF(Tensorflow)도 동일합니다.

- 아래의 코드를 실행하면 데이터셋(Dataset)을 만들수 있습니다.
- 여기에서 NERCorpus는 “튜토리얼 데이터를 원본 문장 + 개체명 태그를 레이블한 문장” 형태로 읽어들이는 역할을 하고요.
- NERDataset는 DataSet 역할을 수행합니다.





In [ ]:
# 코드 6-9 학습 데이터셋 구축
from ratsnlp.nlpbook.ner import NERCorpus, NERDataset
corpus = NERCorpus(args)
train_dataset = NERDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="train",
)

- 이 클래스는 NERCorpus 토크나이저 파트에서 선언해 둔 토크나이저를 품고 있는데요.
- NERCorpus가 넘겨준 데이터(원본 문장, 레이블한 문장)를 모델이 학습할 수 있는 형태로 가공합니다
- 다시 말해 문장을 토큰화하고 이를 인덱스로 변환하는 한편, 레이블한 문장을 모델이 읽어들일 수 있는 포맷으로 바꿔주는 역할을 합니다.

- NERCorpus가 넘겨준 데이터가 다음과 같다고 가정해 봅시다.

```
원본 문장 : ―효진 역의 김환희(14)가 특히 인상적이었다.
레이블한 문장 : ―<효진:PER> 역의 <김환희:PER>(<14:NOH>)가 특히 인상적이었다.
```

- 그러면 NERDataset은 이를 다음과 같은 정보로 변환합니다. tokens는 원본 문장을 토큰화한 뒤 문장 앞뒤에 각각 [CLS]와 [SEP]를 붙이 max_seq_length(=64)가 되도록 패딩 토큰을 추가한 결과입니다.

```
max_seq_length=64, # 토큰 기준 입력 문장 최대 길이
```

```
tokens : [CLS] [UNK] 효 ##진 역 ##의 김 ##환 ##희 ( 14 ) 가 특히 인상 ##적이 ##었다 . [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

labels : [CLS] O B-PER I-PER O O B-PER I-PER I-PER O B-NOH O O O O O O O [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
```



- labels는 레이블한 문장을 tokens에 대응되도록 가공한 결과입니다. 문장 앞뒤에 각각 [CLS]와 [SEP]를 붙이고 코드3의 max_seq_length(=64)가 되도록 패딩 토큰을 추가하는 원칙은 tokens와 같습니다. 레이블한 문장을 보면 개체명은 총 3개가 태깅되어 있는데요.

```
<효진:PER>, <김환희:PER>, <14:NOH>
```
이 바로 그것입니다.

- PER(인명)으로 레이블링된 효진은 tokens 기준 세번째 토큰(효)부터 네번째 토큰(##진)인 걸 확인할 수 있습니다. 이에 labels에는 세번째 토큰과 네번째 토큰이 PER(인명)이 되도록 합니다.

- 단 여기에서 B-는 해당 태그의 시작(Begin), I-는 해당 태그의 시작이 아님(Inside)이라는 뜻을 가집니다.

- PER(인명)으로 레이블링된 김환희는 일곱번째 토큰(김)부터 아홉번째 토큰(##희)인 걸 알 수 있습니다.

- 이에 labels에는 일곱번째 토큰과 아홉번째 토큰이 PER(인명)이 되도록 합니다.

- 마찬가지로 labels의 열한번째 토큰이 NOH(기타 수량표현)이 되도록 만들었습니다.

- 한편 labels에서 O는 outside의 약자로 개체명이 아닌 부분을 의미합니다.

- NERDataset은 여기에 인덱싱 작업을 수행하여 input_ids, attention_mask, token_type_ids, labels를 만듭니다.

- input_ids는 tokens에 인덱싱을 수행한 결과이며 attention_mask는 tokens 각각의 해당 토큰이 패딩인지(0) 아닌지(1)를 나타냅니다.

- token_type_ids는 세그먼트(segment) 정보로 기본값은 모두 0으로 넣습니다.

- label_ids은 labels의 각 개체명 태그(B-PER, I-PER 등)를 정수로 바꾼 결과입니다.

- 개체명 인식을 위한 BERT 모델의 입력은 input_ids, attention_mask, token_type_ids이 되며, 출력은 labels가 되도록 합니다.

- 문장을 모델 입력값으로 변환하는 절차와 관련 자세한 내용은 2장 Preprocess를 참고하시면 좋을 것 같습니다.

```
input_ids : [2, 1, 3476, 4153, 2270, 4042, 420, 4185, 4346, 11, 11524, 12, 197, 9250, 11662, 8805, 8217, 17, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

attention_mask : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

token_type_ids : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

label_ids : [0, 4, 5, 15, 4, 4, 5, 15, 15, 4, 6, 4, 4, 4, 4, 4, 4, 4, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
```
- 실행하면 학습할 때 쓰이는 데이터 로더를 만들 수 있습니다. dataset 역할을 하는 NERDataset은 학습데이터에 속한 각각의 문장을 input_ids, attention_mask, token_type_ids, label_ids 등 네 가지로 변환한 형태로 가지고 있습니다.

- 여기서는 인스턴스(instance)에 해당합니다.

- 데이터 로더는 Dataset이 들고 있는 전체 인스턴스 가운데 배치 크기(코드3에서 정의한 args의 batch_size)만큼을 뽑아 배치 형태로 가공하는 역할을 수행합니다.


In [ ]:
# 코드 6-10 학습 데이터 로더 구축
from torch.utils.data import DataLoader, RandomSampler
train_dataloader = DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    sampler=RandomSampler(train_dataset, replacement=False), # 데이터 로더로 배치 만들때 batch_size 개수만큼 비복원 랜덤 추출
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

- 위의 코드에서 보면 sampler와 collate_fn이 눈에 띕니다. 전자는 샘플링 방식을 정의합니다.
- 6-9 코드 실행으로 만들어진 데이터 로더는 배치를 만들 때 ClassificationDataset이 들고 있는 전체 인스턴스 가운데 batch_size 갯수만큼을 비복원(replacement=False) 랜덤 추출합니다.

- 후자는 이렇게 뽑힌 인스턴스를 배치로 만드는 역할을 하는 함수입니다.
- ClassificationDataset는 파이썬 리스트(list) 형태의 자료형인데요. 이를 파이토치가 요구하는 자료형인 텐서(tensor) 형태로 바꾸는 등의 역할을 수행합니다.



- 코드를 실행하면 평가용 데이터 로더를 구축할 수 있습니다.
- 학습용 데이터 로더와 달리 평가용 데이터 로더는 SequentialSampler를 사용하고 있음을 알 수 있습니다.
```
# SequentialSampler는 batch_size만큼의 갯수만큼을 인스턴스 순서대로 추출하는 역할을 합니다.
```
- 학습때 Batch 구성은 랜덤으로 하는것이 좋음 -> 평가할 때 평가(Test)용 데이터셋 전체를 사용하는데 굳이? 랜덤으로 구성할 이유가 없음.


In [ ]:
# 코드 6-11 평가 데이터 로더 구축
from torch.utils.data import SequentialSampler
val_dataset = NERDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="val",
)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=args.batch_size,
    sampler=SequentialSampler(val_dataset),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

### 6-6. 모델 초기화
- 모델을 초기화합니다.
- BertForTokenClassification은 프리트레인을 마친 BERT 모델 위에 6-1장에서 설명한 개체명 인식을 위한 태스크 모듈이 덧붙여진 형태의 모델 클래스입니다.

```
BertForSequenceClassification - huggingface에서 제공하는 transformer 라이브러리에 있음
```



In [ ]:
# 코드 6-12 모델 초기화
from transformers import BertConfig, BertForTokenClassification
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=corpus.num_labels,
)
model = BertForTokenClassification.from_pretrained(
        args.pretrained_model_name,
        config=pretrained_model_config,
)

### 6-7. 모델 학습 시키기
- 파이토치 라이트닝(pytorch lightning)이 제공하는 라이트닝 모듈(LightningModule) 클래스를 상속받아 태스크(task)를 정의합니다.
- Task에는 모델과 옵티마이저(optimizer), 학습 과정(training process) 등이 정의돼 있습니다.
- 모델은 위의 코드에서 준비한 모델 클래스를 ClassificationTask에 포함시킵니다.
- ClassificationTask 클래스에는 옵티마이저, 러닝레이트 스케줄러(learning rate scheduler)가 정의되어 있는데요.
- 옵티마이저로는 아담(Adam), 러닝레이트 스케줄러로는 ExponentialLR을 사용합니다.

In [ ]:
# 코드 6-13 TASK 정의
from ratsnlp.nlpbook.ner import NERTask
task = NERTask(model, args)

- from ratsnlp.nlpbook.ner import NERTask
task = NERTask(model, args)

In [ ]:
# 코드 6-14 Trainer 정의
trainer = nlpbook.get_trainer(args)

- 트레이너의 fit 함수를 호출하면 학습이 시작됩니다.

In [ ]:
# 코드 6-15 Trainer 정의
trainer.fit(
    task,
    train_dataloader=train_dataloader,
    val_dataloaders=val_dataloader,
)